In [ ]:

# Append main folder
import sys
sys.path.append("../")

import numpy as np 
import torch
from dotmap import DotMap
from neuralg.utils.count_parameters import count_parameters
from neuralg.models.nerf import EigNERF
from neuralg.models.siren import EigSiren
from neuralg.training.train_model import train_model
from neuralg.training.save_run import save_run
from neuralg.utils.set_log_level import set_log_level

d = 5 # 
batch_size = 64
train_matrix_parameters = DotMap({"N":batch_size, 
                            "operation": "eig", 
                            "d": d, 
                            "wigner": True})

run_params = DotMap({ "epoch": 1, # Number of epochs
                           "iterations": 10000, # Batches per epoch
                           "lr": 3e-4} )       # Learning rate

In [ ]:
import time
def estimate_runtime(model,d):
    model.eval()
    N = 1000
    matrices = torch.rand(N,1,1,d,d)
    start_time = time.time()
    for i in range(N):
        model(matrices[i])
    ms_per_matrix = (time.time()-start_time)
    return ms_per_matrix 
def runtime_per_batch(model,d):
    mean = 0
    for i in range(10):
        batch = torch.rand(100,1,d,d)
        start_time = time.time()
        model(batch)
        ms_per_batch = 1000*(time.time()-start_time)
        mean += ms_per_batch
    return mean/10
        

In [ ]:
from neuralg.evaluation.evaluate_model import evaluate_model
from neuralg.evaluation.compute_accuracy import compute_accuracy
from copy import deepcopy
import matplotlib.pyplot as plt
import pickle

file_paths_5 = ['/Users/toveagren/ESA/results/nerf/relunerf5.pk','/Users/toveagren/ESA/results/nerf/sigmnerf5.pk','/Users/toveagren/ESA/results/siren/siren5.pk']
file_paths_10 = ['/Users/toveagren/ESA/results/nerf/relunerf10.pk','/Users/toveagren/ESA/results/siren/siren10.pk']
results = DotMap()
tols = {"5": 0.05 , "10": 0.1}
for file_path in file_paths_5 + file_paths_10:
    # open a file, where you stored the pickled data
    file = open(file_path, 'rb')

    # dump information to that file
    data = pickle.load(file)
    data.runtime = []
    data.batch_runtime = []
    test_parameters = deepcopy(train_matrix_parameters)
    d =  data.models[0].net[-1].out_features
    test_parameters["N"] = 10000
    test_parameters["d"] = d
    tol = tols[str(d)]
    for i, model in enumerate(data.models):
        errors = evaluate_model(model,test_parameters)
        accuracy = compute_accuracy(tol,errors)
        data.accuracy.append(accuracy)
        data.runtime.append(estimate_runtime(model,d))
        data.batch_runtime.append(runtime_per_batch(model,d))

    results[file_path] = data
    

In [ ]:
from matplotlib import cm
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})


fig = plt.figure(facecolor= 'white')
fig.patch.set_facecolor('white')
spec = fig.add_gridspec(1, 2)
fig.suptitle("Trainable parameters ".format(100*tol), fontsize = 30)
ax = fig.add_subplot(spec[0, 0])
for file_path in file_paths_5:
    label = results[file_path].models[0].__class__.__name__
    d =  results[file_path].models[0].net[-1].out_features
    if label == "EigNERF":
        activation = str(results[file_path].models[0].net[0].activation)
        label = label + ', ' + activation
    ax.scatter(results[file_path].no_parameters,results[file_path].accuracy, s = 100, marker='o',label = label)
    ax.set_title("$ {} \\times {}$ Wigner matrices, tol = {}$\%$".format(d,d,tols["5"]*100), fontsize = 18)
    ax.set_xlabel("$\#$ of parameters ", fontsize = 18)
    ax.set_ylabel("Accuracy", fontsize = 18)
    ax.set_xscale('log')
ax.legend(fontsize = 16);

ax = fig.add_subplot(spec[0, 1])
for file_path in file_paths_10:
    label = results[file_path].models[0].__class__.__name__
    d =  results[file_path].models[0].net[-1].out_features
    if label == "EigNERF":
        activation = str(results[file_path].models[0].net[0].activation)
        label = label + ', ' + activation
    ax.scatter(results[file_path].no_parameters,results[file_path].accuracy, s = 100, marker='o',label = label)
    ax.set_title("$ {} \\times {}$ Wigner matrices, tol = {}$\% $ ".format(d,d,tols["10"]*100), fontsize = 18)
    ax.set_xlabel("$\#$ of parameters ", fontsize = 18)
    ax.set_ylabel("Accuracy", fontsize = 18)
    ax.set_xscale('log')
ax.legend(fontsize = 16);
#plt.savefig("trainable_parameters.pdf")

In [ ]:
import torch
fig = plt.figure(facecolor= 'white')
fig.patch.set_facecolor('white')
spec = fig.add_gridspec(1, 2)
layers,neurons = np.arange(5,11,2), [25,50,100,200]
# #,[data.models[0].model_type,data.models[0].net[0].activation]
ax = fig.add_subplot(spec[0, 0])
xgrid, ygrid = np.meshgrid(layers,neurons)
acc5nerf = results[file_paths_5[0]].accuracy
acc5siren = results[file_paths_5[2]].accuracy
fig.suptitle("Network hyperparameters \n Model: EigNERF, Activation: ReLU() ", fontsize = 25)

surf = ax.contourf(xgrid,ygrid, torch.flatten(torch.tensor(acc5nerf)).reshape((3,4)).transpose(0,1), cmap = cm.coolwarm, vmin = 0, vmax = 1)
ax.set_title("$ {} \\times {}$ Wigner matrices ".format(5,5), fontsize = 18)
ax.set_xlabel("$\#$ of hidden layers ", fontsize = 18)
ax.set_ylabel("$\#$ of neurons per layer ", fontsize = 18)
ax.set_xticks(np.arange(5, 11, 2))
ax.set_yticks([25,50,100,200])

fig.colorbar(surf, aspect=15)
plt.tight_layout()

acc10nerf = results[file_paths_10[0]].accuracy
acc10siren = results[file_paths_5[1]].accuracy
ax = fig.add_subplot(spec[0, 1])

surf = ax.contourf(xgrid,ygrid, torch.flatten(torch.tensor(acc10nerf)).reshape((3,4)).transpose(0,1), cmap=cm.coolwarm)
ax.set_title("Accuracy to network hyperparameters ".format(100*tol), fontsize = 18)
ax.set_title("$ {} \\times {}$ Wigner matrices ".format(10,10), fontsize = 18)
ax.set_xlabel("$\#$ of hidden layers ", fontsize = 18)
ax.set_ylabel("$\#$ of neurons per layer ", fontsize = 18)
ax.set_xticks(layers)
ax.set_yticks(neurons)
plt.colorbar(surf, aspect=15)

plt.tight_layout()

#plt.savefig("hyperparameters.pdf")

In [ ]:
plt.rcParams['figure.figsize'] = [20, 10]
fig = plt.figure(facecolor= 'white')
fig.patch.set_facecolor('white')
fig.suptitle("Runtime per single matrix", fontsize = 30)
ax = fig.add_subplot(1,2,1)
file_path = "/Users/toveagren/ESA/results/nerf/relunerf5.pk"
label = results[file_path].models[0].__class__.__name__
d =  results[file_path].models[0].net[-1].out_features
if label == "EigNERF":
    activation = str(results[file_path].models[0].net[0].activation)
    label = label + ', ' + activation
xdata, ydata = results[file_path].runtime,results[file_path].no_parameters
ax.scatter(xdata, acc5nerf, c=acc5nerf, s = 200, cmap=cm.coolwarm)# , label = label);
#ax.scatter(xdata,  acc5siren, c=acc5siren, s = 200, cmap=cm.cool)# , label = label);
ax.set_title(f"$ {d} \\times {d}$ Wigner matrices. Model: " + label , fontsize = 18)
ax.set_xlabel("Runtime per matrix [ms]", fontsize = 20)
ax.set_ylabel("Accuracy", fontsize = 20)
plt.grid(visible=True)
#ax.legend(fontsize = 16);
for i, txt in enumerate(ydata):
    txt = f"({torch.flatten(torch.tensor(xgrid).t())[i]}, {torch.flatten(torch.tensor(ygrid).t())[i]}) \n " + str(txt)
    ax.annotate(txt, (xdata[i] + 0.0015, acc5nerf[i]), fontsize = 20)

print(xdata)

ax = fig.add_subplot(1,2,2)
file_path = "/Users/toveagren/ESA/results/nerf/relunerf10.pk"
label = results[file_path].models[0].__class__.__name__
d =  results[file_path].models[0].net[-1].out_features
if label == "EigNERF":
    activation = str(results[file_path].models[0].net[0].activation)
    label = label + ', ' + activation
xdata, ydata = results[file_path].runtime,results[file_path].no_parameters
ax.scatter(xdata, acc10nerf, c=acc5nerf, s = 200, cmap=cm.coolwarm)# , label = label);
#ax.scatter(xdata,  acc10siren, c=acc5siren, s = 200, cmap=cm.cool)# , label = label);
ax.set_title(f"$ {d} \\times {d}$ Wigner matrices. Model: " + label , fontsize = 18)
ax.set_xlabel("Runtime per matrix [ms]", fontsize = 20)
ax.set_ylabel("Accuracy", fontsize = 20)
plt.grid(visible=True)
#ax.legend(fontsize = 16);
for i, txt in enumerate(ydata):
    txt = f"({torch.flatten(torch.tensor(xgrid).t())[i]}, {torch.flatten(torch.tensor(ygrid).t())[i]}) \n " + str(txt)
    ax.annotate(txt, (xdata[i] + 0.0015, acc10nerf[i]), fontsize = 20)
#print(acc5)
#plt.savefig("../runtime_per_matrix_nerf.pdf")

In [ ]:
fig = plt.figure(facecolor= 'white')
fig.patch.set_facecolor('white')
fig.suptitle("Runtime per single matrix", fontsize = 30)
ax = fig.add_subplot(1,2,1)
file_path = "/Users/toveagren/ESA/results/siren/siren5.pk"
label = results[file_path].models[0].__class__.__name__
d =  results[file_path].models[0].net[-1].out_features
if label == "EigNERF":
    activation = str(results[file_path].models[0].net[0].activation)
    label = label + ', ' + activation
xdata, ydata = results[file_path].runtime,results[file_path].no_parameters
#ax.scatter(xdata, acc5nerf, c=acc5nerf, s = 200, cmap=cm.coolwarm)# , label = label);
ax.scatter(xdata,  acc5siren, c=acc5siren, s = 200, cmap=cm.cool)# , label = label);
ax.set_title(f"$ {d} \\times {d}$ Wigner matrices. Model: " + label , fontsize = 18)
ax.set_xlabel("Runtime per matrix [ms]", fontsize = 20)
ax.set_ylabel("Accuracy", fontsize = 20)
plt.grid(visible=True)
#ax.legend(fontsize = 16);
for i, txt in enumerate(ydata):
    txt = f"({torch.flatten(torch.tensor(xgrid).t())[i]}, {torch.flatten(torch.tensor(ygrid).t())[i]}) \n " + str(txt)
    ax.annotate(txt, (xdata[i] + 0.005, acc5siren[i]-0.03), fontsize = 20)

#print(acc5)

ax = fig.add_subplot(1,2,2)
file_path = "/Users/toveagren/ESA/results/siren/siren10.pk"
label = results[file_path].models[0].__class__.__name__
d =  results[file_path].models[0].net[-1].out_features
if label == "EigNERF":
    activation = str(results[file_path].models[0].net[0].activation)
    label = label + ', ' + activation
xdata, ydata = results[file_path].runtime,results[file_path].no_parameters
#ax.scatter(xdata, acc10nerf, c=acc5nerf, s = 200, cmap=cm.coolwarm)# , label = label);
ax.scatter(xdata,  acc10siren, c=acc10siren, s = 200, cmap=cm.cool)# , label = label);
ax.set_title(f"$ {d} \\times {d}$ Wigner matrices. Model: " + label , fontsize = 18)
ax.set_xlabel("Runtime per matrix [ms]", fontsize = 20)
ax.set_ylabel("Accuracy", fontsize = 20)
plt.grid(visible=True)
#ax.legend(fontsize = 16);
for i, txt in enumerate(ydata):
    layers, neurons= torch.flatten(torch.tensor(xgrid).t())[i], torch.flatten(torch.tensor(ygrid).t())[i]
    txt = f"({layers}, {neurons}) \n " + str(txt)
    if neurons > 25:
        ax.annotate(txt, (xdata[i] + 0.005, acc10siren[i] -0.01), fontsize = 20)
#print(acc5)

In [ ]:

plt.rcParams['figure.figsize'] = [16,16]
fig = plt.figure(facecolor= 'white')
fig.patch.set_facecolor('white')
fig.suptitle("Runtime per batch, batch size = 100", fontsize = 30)
ax = fig.add_subplot(2,1,1)
file_path = "/Users/toveagren/ESA/results/nerf/relunerf5.pk"
label = results[file_path].models[0].__class__.__name__
d =  results[file_path].models[0].net[-1].out_features
if label == "EigNERF":
    activation = str(results[file_path].models[0].net[0].activation)
    label = label + ', ' + activation
x, y = results[file_path].batch_runtime, results[file_path].no_parameters
ax.scatter(x, acc5nerf, c=acc5nerf, s = 200, cmap=cm.coolwarm)# , label = label);
#ax.scatter(xdata,  acc5siren, c=acc5siren, s = 200, cmap=cm.cool)# , label = label);
ax.set_title(f"$ {d} \\times {d}$ Wigner matrices. Model: " + label , fontsize = 18)
ax.set_xlabel("Mean runtime per batch [ms]", fontsize = 20)
ax.set_ylabel("Accuracy", fontsize = 20)
plt.grid(visible=True)
#ax.legend(fontsize = 16);
for i, txt in enumerate(y):
    layers, neurons= torch.flatten(torch.tensor(xgrid).t())[i], torch.flatten(torch.tensor(ygrid).t())[i]
    txt = f"({layers}, {neurons}) \n " + str(txt)
    if neurons > 50:
        ax.annotate(txt, (x[i] + 0.02, acc5nerf[i]-0.01), fontsize = 20)
#print(acc5)

ax = fig.add_subplot(2,1,2)
file_path = "/Users/toveagren/ESA/results/nerf/relunerf10.pk"
label = results[file_path].models[0].__class__.__name__
d =  results[file_path].models[0].net[-1].out_features
if label == "EigNERF":
    activation = str(results[file_path].models[0].net[0].activation)
    label = label + ', ' + activation
xdata, ydata = results[file_path].batch_runtime,results[file_path].no_parameters
ax.scatter(xdata, acc10nerf, c=acc10nerf, s = 200, cmap=cm.coolwarm)# , label = label);
#ax.scatter(xdata,  acc10siren, c=acc5siren, s = 200, cmap=cm.cool)# , label = label);
ax.set_title(f"$ {d} \\times {d}$ Wigner matrices. Model: " + label , fontsize = 18)
ax.set_xlabel("Mean runtime per batch [ms]", fontsize = 20)
ax.set_ylabel("Accuracy", fontsize = 20)
plt.grid(visible=True)
#ax.legend(fontsize = 16);
for i, txt in enumerate(ydata):
    layers, neurons= torch.flatten(torch.tensor(xgrid).t())[i], torch.flatten(torch.tensor(ygrid).t())[i]
    txt = f"({layers}, {neurons}) \n " + str(txt)
    if neurons > 50:
        ax.annotate(txt, (xdata[i] + 0.03, acc10nerf[i] -0.01), fontsize = 20)
print(acc5nerf)
plt.savefig("../runtime_batch.pdf")

In this context, matrices $A \in \mathbf{R}^{5\times 5}$  will be refered to as small, while $B \in \mathbf{R}^{10\times 10}$ are refered to as large.
### Models
Two different architectures are studied in this experiments. They are both densely connected nets. 
#### nerf
For the nerf-model types, skips are made every other layer. 
#### siren 
The siren models have a hyperparameter $\omega_0$. In this experiment, we set $\omega_0$ = 1 (maybe tweek this a little for the larger mdeo)

### Results
#### Trainable parameters
In \cref{fig}, the
#### Hyperparameters
Varying the depth and width of the network evidently impacts prediction performance. 

#### Computational cost
#### Per single matrix  
The runtime for inference on one single matrix is approximated by taking the mean out of 10000 model evaluation runtimes on a random matrix. 
#### Per batch 
We also investigate if there are any differences in runtime results when predicting on batches of matrices. 
### Conclusions
The resulting qualitative trends are mostly ?coherent? for small and large matrices. 

For small matrices, performance of the two different architectures (nerf and siren) are comparable in terms of accuracy per trainable parameter, the nerf-type model having a slight edge. This does not apply to the larger matrices, as even the most parameter rich siren models fail to provide accurate predictions. One should take into consideration that no attention was paid to the hyperparameter $\omega_0$, thus additional performance improvements could be possible if optimized. 

Using relu activation appears more suitable than sigmoid in this application.

In this particular experimental set-up, Fig[insert reference] implies that model performance depends largely on the width of the network, defined as number of neurons per hidden layer, rather than on depth, i.e. number of hidden layers. This implies that increasing performance (Although the training is not very extensive....)

When considering accuracy alone, the EigNERF with 9 hidden layers and 200 neurons per layer proved superior for both small and large matrices, with an accuracy of 0.97 ($5\%$ tolerance) on small matrices and 0.72 ($10\%$ tolerance)  on larger matrices. This is indeed the most parameter-rich model, and Fig [] confirms the increase in computational cost that is commonly associated with larger models. In particular, increasing the depth of the network has a large impact on runtime. Thus, it is also of interest to evaluate accuracy and run time trade-off. For example, the EigNERF with 5 layers and 200 neurons achieved 0.91 accuracy, (6% decrease) but with a single matrix runtime of 0.24 ms (compared to 0.38ms for the largest model) (a 37% decrease). Introducing an efficiency index as $\frac{\text{Accuracy}}{\text{Runtime per matrix}}$, 

A preliminary conclusion is that an efficient model benefits from 

0.2451319694519043, 0.23487305641174316, 0.23038983345031738, 0.24523472785949707, 0.2980022430419922, 0.2886011600494385, 0.29433488845825195, 0.3149130344390869, 0.34850406646728516, 0.35026001930236816, 0.3589670658111572, 0.3855729103088379
 




In [ ]:

fig = plt.figure(facecolor= 'white')
fig.patch.set_facecolor('white')
fig.suptitle("Model efficiency index", fontsize = 30)
ax = fig.add_subplot(2,1,1)
file_path = "/Users/toveagren/ESA/results/nerf/relunerf5.pk"
label = results[file_path].models[0].__class__.__name__
d =  results[file_path].models[0].net[-1].out_features
if label == "EigNERF":
    activation = str(results[file_path].models[0].net[0].activation)
    label = label + ', ' + activation
efficiency = torch.tensor(results[file_path].accuracy)/torch.tensor(results[file_path].runtime)
print(efficiency)

efficiency, indices = torch.sort(efficiency, stable=True)
p = torch.gather(torch.tensor(results[file_path].no_parameters),dim = 0, index=indices)
xgrid_f = torch.gather(torch.flatten(torch.tensor(xgrid).t()),dim = 0, index=indices)
ygrid_f =  torch.gather(torch.flatten(torch.tensor(ygrid).t()),dim = 0, index=indices)

ax.scatter(np.arange(1,13,1), efficiency, c= efficiency, s = 200, cmap=cm.rainbow)# , label = label);
#ax.scatter(xdata,  acc5siren, c=acc5siren, s = 200, cmap=cm.cool)# , label = label);
ax.set_title(f"$ {d} \\times {d}$ Wigner matrices. Model: " + label , fontsize = 18)
#ax.set_xlabel("Mean runtime per batch [ms]", fontsize = 20)
ax.set_ylabel("Accuracy/runtime", fontsize = 20)
ax.grid(visible=True)
ax.set_xticks(np.arange(12,0,-1))
for i, txt in enumerate(p):
    layers, neurons= xgrid_f[i], ygrid_f[i]
    txt = f"({layers}, {neurons}) \n " + str(txt.item())
#     if neurons > 50:
    ax.annotate(txt, (i +1.1, efficiency[i]), fontsize = 20)
# #print(acc5)


ax = fig.add_subplot(2,1,2)
file_path = "/Users/toveagren/ESA/results/nerf/relunerf10.pk"
label = results[file_path].models[0].__class__.__name__
d =  results[file_path].models[0].net[-1].out_features
if label == "EigNERF":
    activation = str(results[file_path].models[0].net[0].activation)
    label = label + ', ' + activation
efficiency = torch.tensor(results[file_path].accuracy)/torch.tensor(results[file_path].runtime)
print(efficiency)

efficiency, indices = torch.sort(efficiency, stable=True)
p = torch.gather(torch.tensor(results[file_path].no_parameters),dim = 0, index=indices)
xgrid_f = torch.gather(torch.flatten(torch.tensor(xgrid).t()),dim = 0, index=indices)
ygrid_f =  torch.gather(torch.flatten(torch.tensor(ygrid).t()),dim = 0, index=indices)

ax.scatter(np.arange(1,13,1), efficiency, c= efficiency, s = 200, cmap=cm.rainbow)# , label = label);
#ax.scatter(xdata,  acc5siren, c=acc5siren, s = 200, cmap=cm.cool)# , label = label);
ax.set_title(f"$ {d} \\times {d}$ Wigner matrices. Model: " + label , fontsize = 18)
#ax.set_xlabel("Mean runtime per batch [ms]", fontsize = 20)
ax.set_ylabel("Accuracy/runtime", fontsize = 20)
ax.set_xticks(np.arange(12,0,-1))
ax.grid(visible= True)
for i, txt in enumerate(p):
    layers, neurons= xgrid_f[i], ygrid_f[i]
    txt = f"({layers}, {neurons}) \n " + str(txt.item())
#     if neurons > 50:
    ax.annotate(txt, (i +1.1, efficiency[i]), fontsize = 20)
# #print(acc5)

#plt.savefig("../modelefficiency_nerf.pdf")

In [ ]:

fig = plt.figure(facecolor= 'white')
fig.patch.set_facecolor('white')
fig.suptitle("Model efficiency index", fontsize = 30)
ax = fig.add_subplot()
file_path = "/Users/toveagren/ESA/results/nerf/relunerf10.pk"
label = results[file_path].models[0].__class__.__name__
d =  results[file_path].models[0].net[-1].out_features
if label == "EigNERF":
    activation = str(results[file_path].models[0].net[0].activation)
    label = label + ', ' + activation
efficiency = torch.tensor(results[file_path].accuracy)/torch.tensor(results[file_path].runtime)
print(efficiency)

efficiency, indices = torch.sort(efficiency, stable=True)
p = torch.gather(torch.tensor(results[file_path].no_parameters),dim = 0, index=indices)
xgrid_f = torch.gather(torch.flatten(torch.tensor(xgrid).t()),dim = 0, index=indices)
ygrid_f =  torch.gather(torch.flatten(torch.tensor(ygrid).t()),dim = 0, index=indices)

ax.scatter(np.arange(12,0,-1), efficiency, c= efficiency, s = 200, cmap=cm.coolwarm)# , label = label);
#ax.scatter(xdata,  acc5siren, c=acc5siren, s = 200, cmap=cm.cool)# , label = label);
ax.set_title(f"$ {d} \\times {d}$ Wigner matrices. Model: " + label , fontsize = 18)
#ax.set_xlabel("Mean runtime per batch [ms]", fontsize = 20)
ax.set_ylabel("Accuracy/runtime", fontsize = 20)
ax.legend(fontsize = 16);
for i, txt in enumerate(p):
    layers, neurons= xgrid_f[i], ygrid_f[i]
    txt = f"({layers}, {neurons}) \n " + str(txt.item())
#     if neurons > 50:
    ax.annotate(txt, (i +1.1, efficiency[-(i+1)]), fontsize = 20)
# #print(acc5)


In [ ]:
# fig = plt.figure(facecolor= 'white')
# fig.patch.set_facecolor('white')
# fig.suptitle("Accuracy to runtime", fontsize = 25)
# ax = fig.add_subplot(projection = '3d')
# file_path = "/Users/toveagren/ESA/results/nerf/relunerf5.pk"
# label = results[file_path].models[0].__class__.__name__
# d =  results[file_path].models[0].net[-1].out_features
# if label == "EigNERF":
#     activation = str(results[file_path].models[0].net[0].activation)
#     label = label + ', ' + activation
# xdata, ydata = results[file_path].runtime,results[file_path].no_parameters
# print(xdata)
# ax.scatter3D(xdata, ydata, acc5nerf, c=acc5nerf, s = 200, cmap=cm.coolwarm, label = label);

# #ax.set_title("$ {} \\times {}$ Wigner matrices, tol = {}$\%$".format(d,d,), fontsize = 18)
# ax.set_xlabel("Runtime [ms]", fontsize = 18)
# ax.set_ylabel("$\#$ of parameters", fontsize = 18)
# ax.legend(fontsize = 16);

# import torch
# from loguru import logger

# training_runs = DotMap({"models": [], "no_parameters": [],  "loss_logs": [], "accuracy": []})

# for hidden_layers in torch.arange(4,11,2):
#     run = DotMap()
#     skip = torch.arange(2,hidden_layers-1,2)
#     for n_neurons in [25,50,100,200]: 
#         #model = EigNERF(d,d**2,n_neurons= n_neurons,skip = skip, hidden_layers=hidden_layers)
#         model = EigSiren(d,hidden_features = n_neurons, hidden_layers=hidden_layers)
#         no_parameters = count_parameters(model)
#         logger.trace(f'Training model with {hidden_layers} hidden layers, {n_neurons} neurons, skipping layer(s) {skip}')
#         training_run = train_model(model,train_matrix_parameters,run_parameters= run_params)
#         training_runs.no_parameters.append(count_parameters(model))
#         training_runs.models.append(training_run.model)
#         training_runs.loss_logs.append(training_run.results)
# save_run(training_runs)